# Physics Informed Neural Networks <br> F1 Car Front Wing Aerodymanics

## PINN

In [1]:
import pandas as pd
import torch
from pinn import PINN
import os

In [2]:
data_dir = "/Users/ggito/repos/pinns/data/front_wing"
model_dir = os.path.join(data_dir, "models")

points_filename = "points_final.csv"
norms_filename = "norms_final.csv"
measurements_filename = "measurements.csv"

In [3]:
if torch.backends.mps.is_available():
  device = torch.device("mps")
elif torch.cuda.is_available():
  device = torch.device("cuda")
else:
  print("GPU device not found.")

print(device)

mps


In [4]:
wing_df = pd.read_csv(os.path.join(data_dir, points_filename))
norm_df = pd.read_csv(os.path.join(data_dir, norms_filename))
measurements_df = pd.read_csv(os.path.join(data_dir, measurements_filename))

In [5]:
# Density (rho): 1.2041kg/m^3
# Dynamic viscosity (mu): 1.81e-5 kg/m.s
rho = 1
mu = 1

# m/s
in_velocity = 1

# Domain limits
x_max = 1
y_max = 1
z_max = 1

In [6]:
input_dim = 3
output_dim = 4
hidden_units = [1000, 1000, 1000]

model_name = "v13"

pinn = PINN(input_dim, output_dim, hidden_units, model_name).to(device)

In [7]:
optimizer = torch.optim.LBFGS(pinn.parameters(), lr=1)

epochs = 2000
checkpoint_epochs = 10

Nf = 100   # num of collocation points -> pde evaluation
Nb = 100   # num of points to evaluate boundary conditions
Nw = 100    # num of points of the surface of the front wing to evaluate boundary conditions
Nu = 35    # num of points of real data

In [8]:
pinn.train_pinn(
    epochs=epochs, 
    optimizer=optimizer, 
    s_df=wing_df, 
    n_df=norm_df, 
    u_df=measurements_df, 
    Nf=Nf, 
    Nb=Nb, 
    Ns=Nw, 
    Nu=Nu, 
    x_max=x_max, 
    y_max=y_max, 
    z_max=z_max, 
    in_velocity=in_velocity, 
    mu=mu, 
    rho=rho, 
    device=device, 
    checkpoint_epochs=checkpoint_epochs, 
    model_dir=model_dir)

PINN(
  (layers): ModuleList(
    (0): Linear(in_features=3, out_features=1000, bias=True)
    (1-2): 2 x Linear(in_features=1000, out_features=1000, bias=True)
    (3): Linear(in_features=1000, out_features=4, bias=True)
  )
)
Model name: v13
Number of epochs: 2000
Number of collocation points Nf: 100
Number of boundary condition points Nb: 100
Number of object surface points Ns: 100
Number of real data points Nu: 35
X max: 1, Y max: 1, Z max: 1
mu: 1, rho: 1
c1: 1.0, c2: 1.0, c3: 1.0, c4: 1.0, c5: 1.0, c6: 1.0, c7: 1.0, c8: 1.0, c9: 1.0, c10: 1.0
Inflow velocity: 1
Device: mps
Checkpoint epochs: 10
Model directory: /Users/ggito/repos/pinns/data/front_wing/models
=> converting Nf, Nb, Ns, Nu to nearest power of 2...
Nf: 128, Nb: 128, Ns: 128, Nu: 32
=> starting training...
Epoch: 1, Total Loss: 8.3141, PDE Loss - Navier Stoker: 0.0062, PDE Loss - Poisson: 0.4384, BC Inlet Loss: 6.3811, BC Outlet Loss: 0.0009, BC Left Loss: 0.0000, BC Right Loss: 0.0000, BC Down Loss: 0.0663, BC Up Los

In [ ]:
# pinn, optimizer = pinn.load_checkpoint_num(optimizer, model_dir, model_name, 50)